In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision.io import ImageReadMode
from torchvision.io import read_image

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, default_collate
from torchsummary import summary
import matplotlib.pyplot as plt

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(device)

def label_images_in_directories(main_directory):
    label_names = []
    image_files = []
    for directory in os.listdir(main_directory):
        sub_directory = os.path.join(main_directory, directory)
        if os.path.isdir(sub_directory):
            for filename in os.listdir(sub_directory):
                image_file = os.path.join(sub_directory, filename)
                if os.path.isfile(image_file) and filename.endswith(".jpg"):
                    label_names.append(directory)
                    image_files.append(image_file)

    image_tensors = [torchvision.io.read_image(image, mode=ImageReadMode.UNCHANGED).to(torch.float32)/255 for image in image_files]
    nr_of_images = len(image_tensors)

    # print(f"The first image has label: {label_names[220]}")
    # print(f"The last image has label: {label_names[nr_of_images - 1]}")
    # print(len(label_names))
    # print(len(image_files))

    return label_names, image_tensors


train_dir = "../data/Train/"
label_names, image_tensors = label_images_in_directories(train_dir)

test_dir = "../data/Test"
label_names, image_tensors = label_images_in_directories(test_dir)


# Define the transformations before entering the neural network
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])

# Load the training and test datasets
train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

# display(train_dataset)
# print(len(train_dataset))
# print(len(test_dataset))

# Create DataLoaders for managing the data batches
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return x


# Create an instance of the model
num_classes = 4
model = CNNModel(num_classes)
model = model.to(device)

summary(model.to(device), input_size=(3,64,64))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 16
totalLoss = []

for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    losses = []

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        
    train_loss = train_loss / len(train_dataset)
    train_acc = train_correct / len(train_dataset)
    totalLoss.append(sum(losses)/len(train_dataset))

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

# Evaluate the model
model.eval()
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()

test_acc = (test_correct / len(test_dataset))*100
print(f"Test Accuracy: {test_acc:.4f}")


fig, ax = plt.subplots()
ax.set(xlabel='Epoch', ylabel='Loss', title="Training Loss")

plt.plot(totalLoss)
plt.show()


cpu


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [12]:
# image resize (224,224) 
# 16 epochs 
# Train Acc = 98
# Test Acc = 69

# image resize (224,224) 
# 16 epochs 
# Train Acc = 96 na eerder 99
# Test Acc = 67

# image resize (224,224) 
# 16 epochs 
# Train Acc = 99
# Test Acc = 61

# image resize (224,224) 
# 16 epochs 
# Train Acc = 99
# Test Acc = 65

# check op overfitten !

# image resize (64,64) 
# 16 epochs 
# Train Acc = 99
# Test Acc = 63